In [38]:
import json
import pandas as pd
from pascal_voc_writer import Writer
import numpy as np
import os
from easydict import EasyDict as edict
import cv2
from pathlib import Path

In [71]:
def load_json(root):
    root = Path(root)
    j = root/'annotations.json'
    x = edict(json.load(open(j)))
    ret = []
    for key, value in x.items():
        filename = value.image
        loc = value.loc
        img = cv2.imread(str(root / 'images' / filename))
        h, w, _ = img.shape
        ret.append(dict(
            x1=loc[0]*w,
            y1=loc[1]*h,
            x2=loc[2]*w,
            y2=loc[3]*h,
            filename=str(root)+'_'+filename,
            clazz='pikachu',
            h=h,
            w=w
        ))
    
    return pd.DataFrame(ret)

In [72]:
train_ann = load_json('train')

val_ann = load_json('val')
train_ann.head(),val_ann.head()

(     clazz             filename    h    w          x1          x2          y1  \
 0  pikachu  train_pikachu_1.png  256  256  154.814606  177.877472  102.899719   
 1  pikachu  train_pikachu_2.png  256  256  107.353424  135.538528   66.457520   
 2  pikachu  train_pikachu_3.png  256  256  115.174576  134.127289  113.109123   
 3  pikachu  train_pikachu_4.png  256  256  144.392685  170.218719  112.707436   
 4  pikachu  train_pikachu_5.png  256  256  135.966843  164.591370   89.815704   
 
            y2  
 0  137.070221  
 1  107.662239  
 2  145.583588  
 3  144.572617  
 4  122.388023  ,
      clazz           filename    h    w          x1          x2          y1  \
 0  pikachu  val_pikachu_1.png  256  256   89.396286  110.914246  101.904282   
 1  pikachu  val_pikachu_2.png  256  256  140.417374  174.420486   85.908524   
 2  pikachu  val_pikachu_3.png  256  256  135.384720  163.016357   81.295319   
 3  pikachu  val_pikachu_4.png  256  256  141.651154  170.642471   88.079170   
 4 

In [77]:
train_file = open('VOC/ImageSets/Main/train.txt', 'w')
val_file = open('VOC/ImageSets/Main/val.txt', 'w')
train_val_file = open('VOC/ImageSets/Main/train_val.txt', 'w')
test_file = open('VOC/ImageSets/Main/test.txt', 'w')

for i in train_ann.filename:
    print(i.split('.')[0], file=train_file, flush=True)
    print(i.split('.')[0], file=train_val_file, flush=True)
for i in val_ann.filename:
    print(i.split('.')[0], file=val_file, flush=True)
    print(i.split('.')[0], file=test_file, flush=True)
    print(i.split('.')[0], file=train_val_file, flush=True)

In [76]:
!rm -rf VOC
!mkdir -p VOC/Annotations
!mkdir -p VOC/ImageSets/Main
!mkdir -p VOC/JPEGImages
# !cp -r train/images/* VOC/JPEGImages/
# !cp -r val/images/* VOC/JPEGImages/

!for name in `ls val/images/*`; do cp ${name} VOC/JPEGImages/val_${name##*/}  ; done
!for name in `ls train/images/*`; do cp ${name} VOC/JPEGImages/train_${name##*/}  ; done
!ls VOC/JPEGImages/ | wc

   1000    1000   21584


In [94]:
!for name in `ls VOC/JPEGImages/*`; do mv ${name} ${name%%.*}.jpg; done

In [93]:
!a=w_learn/d2l-pytorch/data/pikachu/VOC/JPEGImages/train_pikachu_104.png; echo ${a%%.*}.jpg

w_learn/d2l-pytorch/data/pikachu/VOC/JPEGImages/train_pikachu_104.jpg


In [80]:
for i, row in train_ann.iterrows():
    xml_filename=row.filename.split('.')[0] + '.xml'
    xml_filepath='VOC/Annotations/'+xml_filename
    writer = Writer(row.filename, row.w, row.h)
    writer.addObject(row.clazz, row.x1, row.y1, row.x2, row.y2)
    writer.save(xml_filepath)

In [81]:
for i, row in val_ann.iterrows():
    xml_filename=row.filename.split('.')[0] + '.xml'
    xml_filepath='VOC/Annotations/'+xml_filename
    writer = Writer(row.filename, row.w, row.h)
    writer.addObject(row.clazz, row.x1, row.y1, row.x2, row.y2)
    writer.save(xml_filepath)

In [82]:
!zip -q -r pikachu_voc.zip VOC